In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# GPU 가속 라이브러리들
try:
    import cupy as cp
    GPU_AVAILABLE = True
    print("🔥 CuPy 사용 가능 - GPU 가속 활성화!")
except:
    import numpy as cp
    GPU_AVAILABLE = False
    print("⚠️ CuPy 없음 - CPU만 사용")

try:
    import cudf
    import cuml
    from cuml.ensemble import RandomForestClassifier as cuRF
    from cuml.linear_model import LogisticRegression as cuLR
    RAPIDS_AVAILABLE = True
    print("🚀 RAPIDS 사용 가능 - GPU ML 가속!")
except:
    RAPIDS_AVAILABLE = False
    print("⚠️ RAPIDS 없음")

# 기본 라이브러리들
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_classif, RFECV
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import f1_score
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression

import xgboost as xgb
import lightgbm as lgb
import catboost as cb

# 고급 최적화
try:
    import optuna
    from optuna.integration import XGBoostPruningCallback
    OPTUNA_AVAILABLE = True
    print("🎯 Optuna 사용 가능 - 베이지안 최적화!")
except:
    OPTUNA_AVAILABLE = False

# 딥러닝
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.utils.data import Dataset, DataLoader
    PYTORCH_AVAILABLE = True
    print("🧠 PyTorch 사용 가능 - 딥러닝 활성화!")
except:
    PYTORCH_AVAILABLE = False

import time
from multiprocessing import Pool, cpu_count

class H200MonsterKiller:
    """H200 GPU 풀파워 괴물 제압기 - 0.5120+ 목표"""
    
    def __init__(self):
        self.submissions = {}
        self.best_score = 0.5109
        
        # GPU 메모리 정보
        if GPU_AVAILABLE:
            print(f"🔥 GPU 메모리: {cp.cuda.Device().mem_info[1] / 1024**3:.1f} GB")
            
        print("💪 H200 풀파워 모드 활성화!")
        print("🎯 목표: 0.5120+ (괴물 완전 제압!)")
        
    def load_and_ultimate_preprocess(self, random_seed=42):
        """H200 파워로 극한 전처리"""
        np.random.seed(random_seed)
        
        train_df = pd.read_csv('train.csv')
        test_df = pd.read_csv('test.csv')
        
        print(f"  🔬 H200 극한 전처리 시작... (시드: {random_seed})")
        
        feature_cols = [col for col in train_df.columns if col not in ['ID', 'Cancer']]
        
        X_train = train_df[feature_cols].copy()
        y_train = train_df['Cancer'].copy()
        X_test = test_df[feature_cols].copy()
        
        # 1. 기본 전처리
        categorical_cols = X_train.select_dtypes(include=['object']).columns
        
        for col in categorical_cols:
            le = LabelEncoder()
            X_train[col] = le.fit_transform(X_train[col].astype(str))
            
            test_values = X_test[col].astype(str)
            test_encoded = []
            for val in test_values:
                if val in le.classes_:
                    test_encoded.append(le.transform([val])[0])
                else:
                    test_encoded.append(0)
            X_test[col] = test_encoded
        
        # 결측값 처리
        numeric_cols = X_train.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            median_val = X_train[col].median()
            X_train[col].fillna(median_val, inplace=True)
            X_test[col].fillna(median_val, inplace=True)
        
        # 2. 의료 도메인 지식 피처 (고급 버전)
        print("  🏥 고급 의료 도메인 피처 생성...")
        
        # 갑상선암 위험 점수 (의료 논문 기반 가중치)
        X_train['Thyroid_Risk_Score'] = (
            X_train['Family_Background'] * 0.35 +
            X_train['Radiation_History'] * 0.30 +
            X_train['Iodine_Deficiency'] * 0.20 +
            (X_train['Gender'] == 1) * 0.15
        )
        X_test['Thyroid_Risk_Score'] = (
            X_test['Family_Background'] * 0.35 +
            X_test['Radiation_History'] * 0.30 +
            X_test['Iodine_Deficiency'] * 0.20 +
            (X_test['Gender'] == 1) * 0.15
        )
        
        # 호르몬 프로파일 분석
        # TSH 정상 범위: 0.4-4.0 mIU/L
        X_train['TSH_Category'] = 0  # 정상
        X_train.loc[X_train['TSH_Result'] < 0.4, 'TSH_Category'] = -1  # 낮음 (갑상선기능항진)
        X_train.loc[X_train['TSH_Result'] > 4.0, 'TSH_Category'] = 1   # 높음 (갑상선기능저하)
        
        X_test['TSH_Category'] = 0
        X_test.loc[X_test['TSH_Result'] < 0.4, 'TSH_Category'] = -1
        X_test.loc[X_test['TSH_Result'] > 4.0, 'TSH_Category'] = 1
        
        # T4 정상 범위: 5.0-12.0 μg/dL
        X_train['T4_Category'] = 0
        X_train.loc[X_train['T4_Result'] < 5.0, 'T4_Category'] = -1
        X_train.loc[X_train['T4_Result'] > 12.0, 'T4_Category'] = 1
        
        X_test['T4_Category'] = 0
        X_test.loc[X_test['T4_Result'] < 5.0, 'T4_Category'] = -1
        X_test.loc[X_test['T4_Result'] > 12.0, 'T4_Category'] = 1
        
        # T3 정상 범위: 0.8-2.0 ng/dL
        X_train['T3_Category'] = 0
        X_train.loc[X_train['T3_Result'] < 0.8, 'T3_Category'] = -1
        X_train.loc[X_train['T3_Result'] > 2.0, 'T3_Category'] = 1
        
        X_test['T3_Category'] = 0
        X_test.loc[X_test['T3_Result'] < 0.8, 'T3_Category'] = -1
        X_test.loc[X_test['T3_Result'] > 2.0, 'T3_Category'] = 1
        
        # 호르몬 불균형 점수
        X_train['Hormone_Imbalance'] = (
            abs(X_train['TSH_Category']) * 0.4 +
            abs(X_train['T4_Category']) * 0.3 +
            abs(X_train['T3_Category']) * 0.3
        )
        X_test['Hormone_Imbalance'] = (
            abs(X_test['TSH_Category']) * 0.4 +
            abs(X_test['T4_Category']) * 0.3 +
            abs(X_test['T3_Category']) * 0.3
        )
        
        # 3. 고급 피처 엔지니어링 (H200 파워 활용)
        print("  ⚡ H200 파워 고급 피처 생성...")
        
        # 다항식 피처 (2차)
        numeric_features = ['Age', 'Nodule_Size', 'TSH_Result', 'T4_Result', 'T3_Result']
        poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
        
        poly_train = poly.fit_transform(X_train[numeric_features])
        poly_test = poly.transform(X_test[numeric_features])
        
        poly_feature_names = [f'poly_{i}' for i in range(poly_train.shape[1])]
        
        for i, name in enumerate(poly_feature_names):
            X_train[name] = poly_train[:, i]
            X_test[name] = poly_test[:, i]
        
        # 통계적 피처들
        X_train['Hormone_Mean'] = (X_train['TSH_Result'] + X_train['T4_Result'] + X_train['T3_Result']) / 3
        X_train['Hormone_Std'] = ((X_train['TSH_Result'] - X_train['Hormone_Mean'])**2 + 
                                  (X_train['T4_Result'] - X_train['Hormone_Mean'])**2 + 
                                  (X_train['T3_Result'] - X_train['Hormone_Mean'])**2)**0.5
        
        X_test['Hormone_Mean'] = (X_test['TSH_Result'] + X_test['T4_Result'] + X_test['T3_Result']) / 3
        X_test['Hormone_Std'] = ((X_test['TSH_Result'] - X_test['Hormone_Mean'])**2 + 
                                 (X_test['T4_Result'] - X_test['Hormone_Mean'])**2 + 
                                 (X_test['T3_Result'] - X_test['Hormone_Mean'])**2)**0.5
        
        # 4. GPU 가속 특성 선택 (H200 활용)
        if RAPIDS_AVAILABLE:
            print("  🚀 RAPIDS GPU 특성 선택...")
            try:
                # GPU 데이터프레임으로 변환
                X_train_gpu = cudf.from_pandas(X_train)
                y_train_gpu = cudf.from_pandas(y_train)
                
                # GPU 랜덤 포레스트로 특성 중요도 계산
                gpu_rf = cuRF(n_estimators=100, random_state=random_seed)
                gpu_rf.fit(X_train_gpu, y_train_gpu)
                
                # 중요도 상위 특성 선택
                importances = gpu_rf.feature_importances_
                top_features_idx = np.argsort(importances)[-30:]  # 상위 30개
                
                feature_names = X_train.columns
                selected_features = feature_names[top_features_idx]
                
                X_train = X_train[selected_features]
                X_test = X_test[selected_features]
                
                print(f"    GPU 특성 선택 완료: {len(selected_features)}개 특성")
                
            except Exception as e:
                print(f"    RAPIDS 특성 선택 실패: {e}")
        
        return X_train, y_train, X_test, test_df['ID']
    
    def strategy_1_pytorch_deep_learning(self):
        """전략 1: PyTorch 딥러닝 (GPU 가속)"""
        if not PYTORCH_AVAILABLE:
            print("\n❌ PyTorch 없음 - 전략 1 스킵")
            return None
            
        print("\n🧠 전략 1: PyTorch 딥러닝 (H200 GPU 가속)!")
        
        X_train, y_train, X_test, test_ids = self.load_and_ultimate_preprocess(42)
        
        # GPU 설정
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"  디바이스: {device}")
        
        # 데이터 정규화
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # PyTorch 데이터셋 클래스
        class ThyroidDataset(Dataset):
            def __init__(self, X, y=None):
                self.X = torch.FloatTensor(X)
                self.y = torch.LongTensor(y) if y is not None else None
                
            def __len__(self):
                return len(self.X)
                
            def __getitem__(self, idx):
                if self.y is not None:
                    return self.X[idx], self.y[idx]
                return self.X[idx]
        
        # 딥러닝 모델 정의
        class ThyroidNet(nn.Module):
            def __init__(self, input_dim, hidden_dims=[512, 256, 128, 64]):
                super(ThyroidNet, self).__init__()
                
                layers = []
                prev_dim = input_dim
                
                for hidden_dim in hidden_dims:
                    layers.extend([
                        nn.Linear(prev_dim, hidden_dim),
                        nn.BatchNorm1d(hidden_dim),
                        nn.ReLU(),
                        nn.Dropout(0.3)
                    ])
                    prev_dim = hidden_dim
                
                layers.append(nn.Linear(prev_dim, 2))
                self.network = nn.Sequential(*layers)
                
            def forward(self, x):
                return self.network(x)
        
        # 5-fold CV
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        oof_predictions = np.zeros(len(X_train))
        test_predictions = np.zeros(len(X_test))
        
        for fold, (train_idx, val_idx) in enumerate(cv.split(X_train_scaled, y_train)):
            print(f"  Fold {fold+1}/5 훈련 중...")
            
            X_tr, X_val = X_train_scaled[train_idx], X_train_scaled[val_idx]
            y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
            
            # 데이터셋 및 데이터로더
            train_dataset = ThyroidDataset(X_tr, y_tr.values)
            val_dataset = ThyroidDataset(X_val, y_val.values)
            
            train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)
            
            # 모델 초기화
            model = ThyroidNet(X_train_scaled.shape[1]).to(device)
            criterion = nn.CrossEntropyLoss()
            optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
            scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10)
            
            # 훈련
            best_val_loss = float('inf')
            patience_counter = 0
            
            for epoch in range(200):
                # 훈련
                model.train()
                train_loss = 0
                for batch_X, batch_y in train_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                    
                    optimizer.zero_grad()
                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y)
                    loss.backward()
                    optimizer.step()
                    
                    train_loss += loss.item()
                
                # 검증
                model.eval()
                val_loss = 0
                with torch.no_grad():
                    for batch_X, batch_y in val_loader:
                        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                        outputs = model(batch_X)
                        loss = criterion(outputs, batch_y)
                        val_loss += loss.item()
                
                avg_val_loss = val_loss / len(val_loader)
                scheduler.step(avg_val_loss)
                
                # Early stopping
                if avg_val_loss < best_val_loss:
                    best_val_loss = avg_val_loss
                    patience_counter = 0
                    # 최고 모델 저장
                    torch.save(model.state_dict(), f'best_model_fold_{fold}.pth')
                else:
                    patience_counter += 1
                    if patience_counter >= 20:
                        break
            
            # 최고 모델 로드 및 예측
            model.load_state_dict(torch.load(f'best_model_fold_{fold}.pth'))
            model.eval()
            
            with torch.no_grad():
                # OOF 예측
                val_tensor = torch.FloatTensor(X_val).to(device)
                val_outputs = model(val_tensor)
                val_probs = torch.softmax(val_outputs, dim=1)[:, 1].cpu().numpy()
                oof_predictions[val_idx] = val_probs
                
                # 테스트 예측
                test_tensor = torch.FloatTensor(X_test_scaled).to(device)
                test_outputs = model(test_tensor)
                test_probs = torch.softmax(test_outputs, dim=1)[:, 1].cpu().numpy()
                test_predictions += test_probs / 5
        
        # CV 성능 평가
        oof_pred_binary = (oof_predictions > 0.5).astype(int)
        cv_f1 = f1_score(y_train, oof_pred_binary)
        print(f"  ✅ PyTorch CV F1: {cv_f1:.6f}")
        
        # 최종 예측
        final_predictions = (test_predictions > 0.5).astype(int)
        
        submission = pd.DataFrame({'ID': test_ids, 'Cancer': final_predictions})
        self.submissions['pytorch_deep'] = submission
        return submission
    
    def strategy_2_gpu_mega_ensemble(self):
        """전략 2: GPU 메가 앙상블 (100개 모델)"""
        print("\n🔄 전략 2: GPU 메가 앙상블 (100개 모델)!")
        print("  H200 파워로 100개 모델 동시 훈련...")
        
        X_train, y_train, X_test, test_ids = self.load_and_ultimate_preprocess(42)
        
        pos_count = (y_train == 1).sum()
        neg_count = (y_train == 0).sum()
        scale_pos_weight = neg_count / pos_count
        
        # 100개 다양한 모델 생성 (GPU 메모리 활용)
        models = []
        
        print("  100개 모델 생성 중...")
        
        # XGBoost 변형들 (40개)
        for i in range(40):
            seed = 42 + i * 137
            params = {
                'n_estimators': np.random.randint(150, 300),
                'max_depth': np.random.randint(4, 10),
                'learning_rate': np.random.uniform(0.03, 0.15),
                'subsample': np.random.uniform(0.7, 1.0),
                'colsample_bytree': np.random.uniform(0.7, 1.0),
                'reg_alpha': np.random.uniform(0, 1.0),
                'reg_lambda': np.random.uniform(0, 1.0),
                'random_state': seed,
                'scale_pos_weight': scale_pos_weight,
                'tree_method': 'gpu_hist',  # GPU 가속
                'gpu_id': 0,
                'verbosity': 0
            }
            models.append(('xgb_' + str(i), xgb.XGBClassifier(**params)))
        
        # LightGBM 변형들 (30개)
        for i in range(30):
            seed = 42 + i * 239
            params = {
                'n_estimators': np.random.randint(150, 300),
                'max_depth': np.random.randint(4, 10),
                'learning_rate': np.random.uniform(0.03, 0.15),
                'subsample': np.random.uniform(0.7, 1.0),
                'colsample_bytree': np.random.uniform(0.7, 1.0),
                'reg_alpha': np.random.uniform(0, 1.0),
                'reg_lambda': np.random.uniform(0, 1.0),
                'random_state': seed,
                'class_weight': 'balanced',
                'device': 'gpu',  # GPU 가속
                'verbosity': -1
            }
            models.append(('lgb_' + str(i), lgb.LGBMClassifier(**params)))
        
        # CatBoost 변형들 (20개)
        for i in range(20):
            seed = 42 + i * 317
            params = {
                'iterations': np.random.randint(150, 300),
                'depth': np.random.randint(4, 10),
                'learning_rate': np.random.uniform(0.03, 0.15),
                'random_state': seed,
                'task_type': 'GPU',  # GPU 가속
                'verbose': False
            }
            models.append(('cat_' + str(i), cb.CatBoostClassifier(**params)))
        
        # ExtraTrees 변형들 (10개)
        for i in range(10):
            seed = 42 + i * 419
            params = {
                'n_estimators': np.random.randint(200, 400),
                'max_depth': np.random.randint(10, 20),
                'random_state': seed,
                'class_weight': 'balanced',
                'n_jobs': -1
            }
            models.append(('et_' + str(i), ExtraTreesClassifier(**params)))
        
        print(f"  총 {len(models)}개 모델 준비 완료")
        
        # 스태킹 앙상블
        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        oof_predictions = np.zeros((len(X_train), len(models)))
        test_predictions = np.zeros((len(X_test), len(models)))
        
        # 병렬 처리로 모델 훈련
        for i, (name, model) in enumerate(models):
            if i % 20 == 0:
                print(f"  모델 {i+1}/{len(models)} 훈련 중...")
            
            try:
                oof_pred = np.zeros(len(X_train))
                
                for train_idx, val_idx in cv.split(X_train, y_train):
                    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                    
                    model.fit(X_tr, y_tr)
                    oof_pred[val_idx] = model.predict_proba(X_val)[:, 1]
                
                oof_predictions[:, i] = oof_pred
                model.fit(X_train, y_train)
                test_predictions[:, i] = model.predict_proba(X_test)[:, 1]
                
            except Exception as e:
                print(f"    ❌ {name} 실패: {e}")
                oof_predictions[:, i] = 0.12
                test_predictions[:, i] = 0.12
        
        # GPU 메타 모델 (RAPIDS)
        if RAPIDS_AVAILABLE:
            try:
                print("  🚀 RAPIDS GPU 메타모델 훈련...")
                oof_gpu = cudf.from_pandas(pd.DataFrame(oof_predictions))
                y_gpu = cudf.from_pandas(y_train)
                
                meta_model = cuLR(class_weight='balanced')
                meta_model.fit(oof_gpu, y_gpu)
                
                test_gpu = cudf.from_pandas(pd.DataFrame(test_predictions))
                final_proba = meta_model.predict_proba(test_gpu)[:, 1].to_array()
                
            except Exception as e:
                print(f"    RAPIDS 메타모델 실패: {e}, sklearn 사용")
                meta_model = LogisticRegression(random_state=42, class_weight='balanced', C=0.1)
                meta_model.fit(oof_predictions, y_train)
                final_proba = meta_model.predict_proba(test_predictions)[:, 1]
        else:
            meta_model = LogisticRegression(random_state=42, class_weight='balanced', C=0.1)
            meta_model.fit(oof_predictions, y_train)
            final_proba = meta_model.predict_proba(test_predictions)[:, 1]
        
        final_predictions = (final_proba > 0.5).astype(int)
        
        submission = pd.DataFrame({'ID': test_ids, 'Cancer': final_predictions})
        self.submissions['gpu_mega_ensemble'] = submission
        
        print("  ✅ 100개 모델 GPU 앙상블 완료!")
        return submission
    
    def strategy_3_optuna_extreme_optimization(self):
        """전략 3: Optuna 극한 최적화 (1000회 시도)"""
        if not OPTUNA_AVAILABLE:
            print("\n❌ Optuna 없음 - 전략 3 스킵")
            return None
            
        print("\n🎯 전략 3: Optuna 극한 최적화 (1000회 시도)!")
        print("  H200 파워로 1000회 베이지안 최적화...")
        
        X_train, y_train, X_test, test_ids = self.load_and_ultimate_preprocess(42)
        
        pos_count = (y_train == 1).sum()
        neg_count = (y_train == 0).sum()
        scale_pos_weight = neg_count / pos_count
        
        def objective(trial):
            # 매우 세밀한 파라미터 탐색
            params = {
                'n_estimators': trial.suggest_int('n_estimators', 100, 500),
                'max_depth': trial.suggest_int('max_depth', 3, 12),
                'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.3, log=True),
                'subsample': trial.suggest_float('subsample', 0.5, 1.0),
                'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
                'reg_alpha': trial.suggest_float('reg_alpha', 0, 2.0),
                'reg_lambda': trial.suggest_float('reg_lambda', 0, 2.0),
                'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
                'gamma': trial.suggest_float('gamma', 0, 2.0),
                'random_state': 42,
                'scale_pos_weight': scale_pos_weight,
                'tree_method': 'gpu_hist',
                'gpu_id': 0,
                'verbosity': 0
            }
            
            model = xgb.XGBClassifier(**params)
            
            # 빠른 CV (3-fold)
            cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
            cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring='f1', n_jobs=1)
            
            return cv_scores.mean()
        
        try:
            # Optuna 연구 생성
            study = optuna.create_study(
                direction='maximize',
                pruner=optuna.pruners.MedianPruner(n_startup_trials=50, n_warmup_steps=10)
            )
            
            # 1000회 최적화 (H200 파워로!)
            study.optimize(objective, n_trials=1000, timeout=3600)  # 1시간 한도
            
            best_params = study.best_params
            best_score = study.best_value
            
            print(f"  ✅ Optuna 최고 점수: {best_score:.6f}")
            print(f"  최적 파라미터: {best_params}")
            
            # 최적 파라미터로 스태킹 앙상블
            models = {
                'xgb_optimized': xgb.XGBClassifier(**best_params),
                'lgb_baseline': lgb.LGBMClassifier(n_estimators=200, max_depth=6, learning_rate=0.08, random_state=42, class_weight='balanced', device='gpu', verbosity=-1),
                'cat_baseline': cb.CatBoostClassifier(iterations=200, depth=6, learning_rate=0.08, random_state=42, task_type='GPU', verbose=False)
            }
            
            cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            oof_predictions = np.zeros((len(X_train), len(models)))
            test_predictions = np.zeros((len(X_test), len(models)))
            
            for i, (name, model) in enumerate(models.items()):
                print(f"  {name} 훈련 중...")
                oof_pred = np.zeros(len(X_train))
                
                for train_idx, val_idx in cv.split(X_train, y_train):
                    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
                    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
                    
                    model.fit(X_tr, y_tr)
                    oof_pred[val_idx] = model.predict_proba(X_val)[:, 1]
                
                oof_predictions[:, i] = oof_pred
                model.fit(X_train, y_train)
                test_predictions[:, i] = model.predict_proba(X_test)[:, 1]
            
            # 메타 모델
            meta_model = LogisticRegression(random_state=42, class_weight='balanced', C=0.1)
            meta_model.fit(oof_predictions, y_train)
            
            final_proba = meta_model.predict_proba(test_predictions)[:, 1]
            final_predictions = (final_proba > 0.5).astype(int)
            
            submission = pd.DataFrame({'ID': test_ids, 'Cancer': final_predictions})
            self.submissions['optuna_extreme'] = submission
            return submission
            
        except Exception as e:
            print(f"  ❌ Optuna 극한 최적화 실패: {e}")
            return None
    
    def strategy_4_h200_ultimate_fusion(self):
        """전략 4: H200 궁극 퓨전"""
        print("\n👹 전략 4: H200 궁극 퓨전!")
        
        if len(self.submissions) < 2:
            print("  충분한 전략이 실행되지 않음")
            return None
        
        # 가중치 (GPU 기반 전략들에 더 높은 가중치)
        strategy_weights = {
            'pytorch_deep': 1.5,        # 딥러닝
            'gpu_mega_ensemble': 1.6,   # 가장 높은 가중치
            'optuna_extreme': 1.4,      # 극한 최적화
        }
        
        weights = []
        predictions = []
        
        for name, submission in self.submissions.items():
            if name in strategy_weights:
                predictions.append(submission['Cancer'].values)
                weights.append(strategy_weights[name])
                print(f"  {name}: 가중치 {strategy_weights[name]}")
        
        if len(predictions) == 0:
            print("  유효한 전략이 없음")
            return None
        
        # 가중 평균
        weighted_avg = np.average(predictions, axis=0, weights=weights)
        
        # 임계값 미세 조정
        thresholds = np.arange(0.45, 0.55, 0.001)
        best_threshold = 0.5
        
        for threshold in thresholds:
            final_predictions = (weighted_avg >= threshold).astype(int)
            pred_ratio = final_predictions.mean()
            
            # 12% ± 0.3% 범위 내에서 선택
            if abs(pred_ratio - 0.12) < 0.003:
                best_threshold = threshold
                print(f"  ✅ 임계값 {threshold:.3f} 선택 (비율: {pred_ratio:.4f})")
                break
        
        final_predictions = (weighted_avg >= best_threshold).astype(int)
        
        test_ids = list(self.submissions.values())[0]['ID']
        submission = pd.DataFrame({'ID': test_ids, 'Cancer': final_predictions})
        
        self.submissions['h200_ultimate'] = submission
        return submission

def run_h200_monster_hunt():
    """H200 괴물 사냥 실행"""
    print("👹 H200 GPU 풀파워 괴물 사냥!")
    print("=" * 60)
    print("🔥 NVIDIA H200 + 140GB 메모리 풀가동!")
    print("🎯 목표: 0.5120+ (괴물 완전 제압!)")
    print("⚡ GPU 가속 딥러닝 + 메가 앙상블 + 극한 최적화")
    print("=" * 60)
    
    start_time = time.time()
    killer = H200MonsterKiller()
    
    strategies = [
        ("PyTorch 딥러닝", killer.strategy_1_pytorch_deep_learning),
        ("GPU 메가 앙상블", killer.strategy_2_gpu_mega_ensemble),
        ("Optuna 극한 최적화", killer.strategy_3_optuna_extreme_optimization),
        ("H200 궁극 퓨전", killer.strategy_4_h200_ultimate_fusion)
    ]
    
    for i, (name, strategy_func) in enumerate(strategies, 1):
        print(f"\n{'='*20} {name} {'='*20}")
        strategy_start = time.time()
        
        try:
            result = strategy_func()
            if result is not None:
                filename = f'h200_kill_{i}.csv'
                result.to_csv(filename, index=False)
                print(f"  💾 저장: {filename}")
        except Exception as e:
            print(f"  ❌ {name} 실패: {e}")
        
        strategy_time = time.time() - strategy_start
        print(f"  ⏱️ 소요 시간: {strategy_time:.1f}초")
    
    # 최종 H200 킬러
    if 'h200_ultimate' in killer.submissions:
        best_submission = killer.submissions['h200_ultimate']
        best_submission.to_csv('H200_MONSTER_DESTROYER.csv', index=False)
        print(f"\n👹 H200 괴물 파괴자: H200_MONSTER_DESTROYER.csv")
    
    total_time = time.time() - start_time
    
    print(f"\n🏆 H200 작전 완료!")
    print(f"⏱️ 총 소요 시간: {total_time:.1f}초")
    print(f"🔥 GPU 파워: 100% 활용")
    print(f"📁 생성된 파일:")
    print(f"  1. H200_MONSTER_DESTROYER.csv ⭐ (최종 보스)")
    print(f"  2. h200_kill_2.csv (100개 모델 메가 앙상블)")
    print(f"  3. h200_kill_1.csv (PyTorch 딥러닝)")
    print(f"  4. h200_kill_3.csv (1000회 Optuna)")
    
    print(f"\n🎯 기대 결과:")
    print(f"  - 0.5112+: GPU 딥러닝 효과!")
    print(f"  - 0.5116+: 메가 앙상블 파워!")
    print(f"  - 0.5120+: 진짜 괴물 완전 제압! 👹💀")
    
    print(f"\n🔥 H200의 진짜 파워를 보여줬습니다!")
    print(f"💪 140GB 메모리 + GPU 가속으로 모든 걸 시도했습니다!")

if __name__ == "__main__":
    run_h200_monster_hunt()

⚠️ CuPy 없음 - CPU만 사용
⚠️ RAPIDS 없음
🧠 PyTorch 사용 가능 - 딥러닝 활성화!
👹 H200 GPU 풀파워 괴물 사냥!
🔥 NVIDIA H200 + 140GB 메모리 풀가동!
🎯 목표: 0.5120+ (괴물 완전 제압!)
⚡ GPU 가속 딥러닝 + 메가 앙상블 + 극한 최적화
💪 H200 풀파워 모드 활성화!
🎯 목표: 0.5120+ (괴물 완전 제압!)

==================== PyTorch 딥러닝 ====================

🧠 전략 1: PyTorch 딥러닝 (H200 GPU 가속)!
  🔬 H200 극한 전처리 시작... (시드: 42)
